In [1]:
import pandas as pd
import anndata

import scanpy as sc

In [2]:
adata = sc.read('../data/adata_acini.h5ad')

In [3]:
for ncell in range(4, 21, 4):
    new_colnm = 'mask_ncell{}'.format(ncell)
    adata.obs[new_colnm] = '-1'

In [4]:
for sid in adata.obs.Batch.unique():
    fn = '../results/dfs_newlyassigned_rescaled2x/adata_cols_expand_{}.csv'.format(sid)
    if os.path.isfile(fn):
        tmp = pd.read_csv(fn, index_col=0)
        tmp = tmp.astype('str')
        for cnm in tmp.columns:
            adata.obs.loc[tmp.index, cnm] = tmp[cnm]

In [ ]:
adata

In [8]:
df = []
for sid in adata.obs.Batch.unique():
    fn = '../results/mask_coords_area/lummask_area_{}.csv'.format(sid)
    if os.path.isfile(fn):
        tmp = pd.read_csv(fn, index_col=0)
        tmp = tmp.transpose()
        if len(df) == 0:
            df = tmp.copy()
        else:
            df = pd.concat([df, tmp])


In [9]:
df.columns = ['mask_luminal']

In [10]:
df_area2 = df.copy()

In [11]:
tmpdf = []
for sid in adata.obs.Batch.unique():
    fn = '../results/dfs_newlyassigned_rescaled2x/area_mask_afterdil_{}.csv'.format(sid)
    if os.path.isfile(fn):
        tmp = pd.read_csv(fn, index_col=0)
        if len(tmpdf) == 0:
            tmpdf = tmp.copy()
        else:
            tmpdf = pd.concat([tmpdf, tmp])
df_area2['mask_afterdil'] = tmpdf['mask_afterdil']

In [12]:
for d_ncell in range(2, 21, 2):
    new_colnm = 'mask_ncell{}'.format(d_ncell)
    tmpdf = []
    for sid in adata.obs.Batch.unique():
        fn = '/common/yangy4/xenium/tunji_rankl/adatas/dfs_newlyassigned_rescaled2x/area_{}_{}.csv'.format(sid, new_colnm)
        if os.path.isfile(fn):
            tmp = pd.read_csv(fn, index_col=0)
            if len(tmpdf) == 0:
                tmpdf = 10*tmp.copy()
            else:
                tmpdf = pd.concat([tmpdf, 10*tmp])
    df_area2[new_colnm] = tmpdf[new_colnm]

In [13]:
df_area = df_area2.copy()

In [14]:
df_area['sid'] = [a.split('s1835')[-1] for a in df_area.index]

In [ ]:
adata.write('../data/adata_final.h5ad')

In [5]:
cnms = ['acini_nodil', 'acini_mask', 'acini_dist4_dist', 'acini_dist4', 'acini_dist8_dist', 'acini_dist8', 'acini_dist12_dist', 'acini_dist12',
        'acini_dist16_dist', 'acini_dist16', 'acini_dist20_dist', 'acini_dist20', 'acini_dist24_dist', 'acini_dist24', 'acini_dist28_dist', 'acini_dist28',
        'acini_dist32_dist', 'acini_dist32', 'acini_dist36_dist', 'acini_dist36', 'acini_dist40_dist', 'acini_dist40']
for c in cnms:
    del adata.obs[c]

In [6]:
adata

AnnData object with n_obs × n_vars = 119036 × 380
    obs: 'Batch', 'source', 'prediction', 'n_genes', 'doublet_score', 'x_centroid', 'y_centroid', 'z_centroid', 'area', 'x_centroid_px', 'y_centroid_px', 'n_genes_by_counts', 'total_counts', 'leiden_label', 'Type', 'dapi_mask', 'ACTA2_mask', 'EPCAM_mask', 'label_mask', 'PID', 'mask_luminal', 'mask_afterdil', 'mask_ncell2', 'mask_ncell4', 'mask_ncell6', 'mask_ncell8', 'mask_ncell10', 'mask_ncell12', 'mask_ncell14', 'mask_ncell16', 'mask_ncell18', 'mask_ncell20'
    var: 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: 'dendrogram_leiden_label', 'leiden', 'log1p', 'neighbors', 'prediction_colors', 'scrublet', 'source_colors', 'umap', 'wilcoxon_leiden'
    obsm: 'X_pca', 'X_scANVI', 'X_scANVI_harmony', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'